In [ ]:
import tensorflow as tf
import qvgg16
import qresnet
import dataset
from tensorflow import keras
from keras import layers
from qkeras import *

# load the data

In [ ]:
x_train, y_train, x_test, y_test = dataset.load_cifar100()

# create the model

In [ ]:
x = x_in = layers.Input(x_train.shape[1:], name="input")
x = keras.layers.RandomFlip(mode="horizontal")(x)
# x = keras.layers.RandomBrightness(factor=(-0.1,0.1))(x)
# x = keras.layers.RandomContrast(factor=0.1)(x)
x = keras.layers.GaussianNoise(0.05)(x)
x = QActivation(quantized_relu_po2(4,1,use_stochastic_rounding=True))(x)
x = qvgg16.VGG_16(x, num_classes=100)
# x = qresnet.resnet32(x, num_classes=100, width_factor=2.0)
qmodel = keras.Model(inputs=[x_in], outputs=[x])
qmodel.summary()

# compile

In [ ]:
class ResNetPaperLR(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr = 0.1, learning_rate_decay=10, steps=[32000, 48000]):
        super(ResNetPaperLR, self).__init__()
        self.initial_lr = initial_lr
        self.learning_rate_decay = learning_rate_decay
        self.steps = steps
        self.current_lr = 0
    def __call__(self, step):        
        step12 = tf.where(step < self.steps[1], self.initial_lr/self.learning_rate_decay, self.initial_lr/(self.learning_rate_decay**2))    
        step01 = tf.where(step < self.steps[0], self.initial_lr, step12)
        return step01
                         

    def get_config(self):
        return {
            "initial_lr": self.initial_lr,
            "learning_rate_decay": self.learning_rate_decay,
            "steps": self.steps
        }


NB_EPOCH = 1
BATCH_SIZE = 128
VERBOSE = 1
STEPS = int((x_train.shape[0] / BATCH_SIZE) * NB_EPOCH)
OPTIMIZER = keras.optimizers.Adam(learning_rate=ResNetPaperLR(0.0001, steps=[int(STEPS*0.6), int(STEPS*0.9)]), decay=0.000025)
qmodel.compile(optimizer=OPTIMIZER, loss=keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.02), metrics=["accuracy"])

# train

In [ ]:
history = qmodel.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, initial_epoch=0, verbose=VERBOSE, validation_data=(x_test, y_test), validation_freq=2)

In [ ]:
NB_EPOCH = 25
BATCH_SIZE = 128
VERBOSE = 1
STEPS = int((x_train.shape[0] / BATCH_SIZE) * NB_EPOCH)
OPTIMIZER = keras.optimizers.Adam(learning_rate=ResNetPaperLR(0.001, steps=[int(STEPS*0.6), int(STEPS*0.9)]), decay=0.000025)
qmodel.compile(optimizer=OPTIMIZER, loss=keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.02), metrics=["accuracy"])

In [ ]:
history = qmodel.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, initial_epoch=0, verbose=VERBOSE, validation_data=(x_test, y_test), validation_freq=2)

In [ ]:
import matplotlib.pyplot as plt


#plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()    
#plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.show()    

# save the model

In [ ]:
qmodel.save("vggmodels/cifar_fp32.h5")

# load the model and quantize

In [ ]:
from qkeras.utils import model_quantize

quantizer_config = {    
    "QConv2D": {
        "kernel_quantizer": "quantized_po2(4,1,use_stochastic_rounding=True)",
        "bias_quantizer": "quantized_po2(4,1,use_stochastic_rounding=True)"
    },
    "QDense": {
        "kernel_quantizer": "quantized_po2(4,1,use_stochastic_rounding=True)",
        "bias_quantizer": "quantized_po2(4,1,use_stochastic_rounding=True)"
    },
    "QActivation": { "relu": "quantized_relu_po2(4,1,use_stochastic_rounding=True)" },    
}

qmodel2 = model_quantize(qmodel, quantizer_config, activation_bits=4, transfer_weights=True)

for layer in qmodel2.layers:
    if hasattr(layer, "kernel_quantizer"):
        print(layer.name, "kernel:", str(layer.kernel_quantizer_internal), "bias:", str(layer.bias_quantizer_internal))
    elif hasattr(layer, "quantizer"):
        print(layer.name, "quantizer:", str(layer.quantizer))

# finetune the quantized model

In [ ]:
NB_EPOCH = 8
BATCH_SIZE = 128
VERBOSE = 1
STEPS = int((x_train.shape[0] / BATCH_SIZE) * NB_EPOCH)

OPTIMIZER = keras.optimizers.Adam(learning_rate=ResNetPaperLR(0.0001, steps=[int(STEPS*0.6), int(STEPS*0.9)]), decay=0.000025)

qmodel2.compile(optimizer=OPTIMIZER, loss=keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.02), metrics=["accuracy"])
history = qmodel2.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, initial_epoch=0, verbose=VERBOSE, validation_data=(x_test, y_test), validation_freq=2)

# save the model

In [ ]:
from qkeras.utils import model_save_quantized_weights, load_qmodel


dic = model_save_quantized_weights(qmodel2, "qmodels/cifar100_vgg16/qmodel_weights")
dic = model_save_quantized_weights(qmodel2, "qmodels/cifar100_vgg16/qmodel_weights.h5")
qmodel2.save("qmodels/cifar100_vgg16/model.h5")
qmodel_load_test = load_qmodel("qmodels/cifar100_vgg16/model.h5", custom_objects={"ResNetPaperLR":ResNetPaperLR})
qmodel_load_test.evaluate(x_test, y_test)


In [ ]:
print_qstats(qmodel2)